<div class="alert alert-block alert-success">
    <h1 align="center">Representation Learning using DAE and Swap Noise</h1>
    <h3 align="center"> Tabular Time Series</h3>
    <h5 align="center">Github: (https://github.com/MTisMT)</h5>
</div>

Appling an unsupervised denoising auto encoder to independent variables 
for reaching a better representation of the features. both train & test data should be used for this part;
this technique is based on Porto Seguro’s Safe Driver Prediction kaggle competition
 [Kaggle winning solution](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/44629)

## Import libaries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Input, BatchNormalization, LSTM

## Adding swap noise to tabular data

In [ ]:
def add_swap_noise(data,perc=0.15,pandas_df=True):
    row=data.shape[0]
    col=data.shape[1]
    per=perc
    num=int(col*per)
    data_noisy=data.copy()
    rnd_row=np.random.permutation(row)
    for i in range(data.shape[0]):
        temp=np.random.permutation(col)
        for j in range(num):
            if pandas_df:
                data_noisy.iloc[i,int(temp[j])] = data.iloc[rnd_row[i],int(temp[j])]
            else:
                data_noisy[i,int(temp[j])] = data[rnd_row[i],int(temp[j])]
    return data_noisy

## Denoising Auto-Encoder (DAE)

In [5]:
# Input of NN: X_noisy
# Output of NN: X
def AutoEn(X_noisy, X, val_size=0.8, compress_ratio=2, ep=100, lr=1e-5,bs=32,min_lr=1e-7,f_lr=0.2):
    tr_size =int((1-val_size) * X.shape[0])
    input_sz = X.shape[1]
    model = Sequential()
    model.add(Input(shape=(input_sz,)))
    model.add(Dense(1000))
    model.add(Dense(input_sz//compress_ratio))
    model.add(Dense(1000,activation='tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(1000,activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(input_sz,activation='sigmoid'))
    opt = optimizers.Adam(learning_rate=lr)
    #opt = optimizers.SGD(learning_rate=lr, momentum=0.9)
    model.compile(optimizer=opt,loss='mean_squared_error')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=f_lr,
                              patience=10, min_lr=min_lr)
    ES = EarlyStopping(monitor='loss', restore_best_weights=True, patience=10)
    model.hist = model.fit(X_noisy[:tr_size],X[:tr_size],epochs=ep, callbacks=[ES,reduce_lr],
              validation_data=(X_noisy[tr_size:],X[tr_size:]),shuffle=False, batch_size=bs,verbose=1)
    return model

## Representation Learning Features
Using Output of DAE

In [ ]:
def DAE_rep(data, DAE_model):
    data_noisy = add_swap_noise(data,perc=0.15)
    DAE = DAE_model(data_noisy, data, val_size=0.2 ,comp_r=1.5, ep=100, lr=5e-3, bs=50, min_lr=1e-8, f_lr=0.25)
    rep_data = DAE.predict(data_noisy)
    return rep_data

## Representation Learning Features
Using values of the activations on all hidden layers

In [6]:
# To be written